In [125]:
from __future__ import  absolute_import
from __future__ import  division
import torch as t
from data.voc_dataset import VOCBboxDataset
from skimage import transform as sktsf
from torchvision import transforms as tvtsf
from data import util
import numpy as np
from utils.config import opt

def normalize(img):
    """
    return appr -125-125 BGR
    """
    img = img[[2, 1, 0], :, :]  # RGB-BGR
    mean = np.array([122.7717, 115.9465, 102.9801]).reshape(3, 1, 1)

    img = (img - mean).astype(np.float32, copy=True)
    return img

class Dataset:
    def __init__(self, opt, split='train', img_type='png'):
        self.opt = opt
        self.db = VOCBboxDataset(opt.voc_data_dir, split=split, img_type=img_type)

    def __getitem__(self, idx):
        ori_img, bbox, label, difficult = self.db.get_example(idx)
#         img = normalize(ori_img)
        img = ori_img
    
        return img.copy()

    def __len__(self):
        return len(self.db)

In [126]:
def calc_means_stdev(dataset):
    channel_sums = np.zeros((3))
    sq_channel_sums = np.zeros((3))
    n = 0

    for i in range(len(dataset)):
        img = dataset[i]
        img_sq = img**2
        c, w, h = img.shape

        n += w * h
        channel_sums += np.sum(img, axis=(1,2))
        sq_channel_sums += np.sum(img_sq, axis=(1,2))
    
    means = channel_sums / n
    variance = ((n*sq_channel_sums) - (channel_sums**2)) / (n*n)
    stdev = np.sqrt(variance)
    
    return means, stdev

## KITTI train

In [129]:
opt.voc_data_dir = '/media/tadenoud/DATADisk/datasets/kitti_2d/VOC2012/'
dataset = Dataset(opt, split='train')
kitti_means, kitti_stdev = calc_means_stdev(dataset)

print("BGR means", kitti_means[::-1])
print("BGR stdev", kitti_stdev[::-1])

BGR means [93.58779838 97.81602497 92.86639532]
BGR stdev [80.9433865  79.54839174 78.36423657]


## PascalVOC trainval

In [130]:
opt.voc_data_dir = '/home/tadenoud/Git/simple-faster-rcnn-pytorch/VOCdevkit/VOC2007/'
dataset = Dataset(opt, split='trainval', img_type='jpg')
pascal_means, pascal_stdev = calc_means_stdev(dataset)

print("BGR means", pascal_means[::-1])
print("BGR stdev", pascal_stdev[::-1])

BGR means [ 99.89283101 108.29856793 114.35074658]
BGR stdev [72.46183777 69.24872388 70.12118565]


## Calculate Kitti correction factor

In [ ]:
old_means = np.array([122.7717, 115.9465, 102.9801])